In [5]:
import pandas as pd

df_all = pd.DataFrame()

for year in range(2017, 2021):
    season = str(year) + '-' + str(year+1)[-2:]
    csvname = 'input/xg_bundesliga_' + season + '.csv'

    df = pd.read_csv(csvname)[['Wk', 'Home', 'xG', 'Score', 'xG.1', 'Away']].rename(columns={'Wk': 'round',
                                                                                             'Home': 'team1',
                                                                                             'xG': 'xg1',
                                                                                             'xG.1': 'xg0',
                                                                                             'Away': 'team0'
                                                                                            }).dropna()

    df['season'] = season
    df['round'] = df['round'].astype(int)                                                                                                     
    df['score1'] = df['Score'].str[0].astype(int)
    df['score0'] = df['Score'].str[2].astype(int)
    df = df.drop(columns="Score")

    df['xg_against1'] = df['xg0']
    df['xg_against0'] = df['xg1']
    df['score_against1'] = df['score0']
    df['score_against0'] = df['score1']

    df = df.reset_index().rename(columns={'index': 'match_id'})
    df = pd.wide_to_long(df, ['team', 'xg', 'xg_against', 'score', 'score_against'], i='match_id', j='home').sort_values(by=['match_id', 'home'], ascending=[True, False]).reset_index()

    df_all = df_all.append(df, ignore_index=True)


### Export data
df_all.to_csv('input/xg_bundesliga.csv', index=False)  
df_all[['team']].rename(columns={'team': 'team_xg'}).groupby('team_xg').count().to_csv('input/team_names_xg.csv')
